# Notebook version of NSGA-II constrained, uses xopt.nsga2

In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [1]:
from xopt import nsga2

from deap import base, tools, creator
from deap.benchmarks.tools import diversity, convergence, hypervolume

import numpy as np

import random

# Test problems

In [ ]:
NAME = 'CONSTR'
BOUND_LOW, BOUND_UP = [0.1, 0] , [1, 5]
WEIGHTS = [-1, -1] # Minimize
NDIM = 2
N_CONSTRAINTS = 2

def CONSTR(individual):
    #time.sleep(.01)
    x1=individual[0]
    x2=individual[1]
    objectives =  (x1, (1.0+x2)/x1)
    constraints = (x2+9*x1-6.0, -x2+9*x1-1.0) # Should be >= 0 
    return (objectives, constraints)

F = CONSTR
# For plotting
X_RANGE = [0.1, 1]
Y_RANGE = [0, 10]

In [3]:
NAME = 'TNK'
BOUND_LOW, BOUND_UP = [0.0, 0.0], [3.14159, 3.14159]  
WEIGHTS = [-1, -1] # Minimize
NDIM = 2
N_CONSTRAINTS = 2

def TNK(individual):    
    x1=individual[0]
    x2=individual[1]
    objectives =  (x1, x2)
    constraints = (x1**2+x2**2-1.0 - 0.1*np.cos(16*np.arctan2(x1, x2)), 0.5-(x1-0.5)**2-(x2-0.5)**2 )
    return (objectives, constraints, (x1, x2))

F = TNK
X_RANGE = [0, 1.4]
Y_RANGE = [0, 1.4]

In [ ]:
NAME = 'SRN'
BOUND_LOW, BOUND_UP = [-20.0, -20.0], [20.0, 20.0]  
WEIGHTS = [-1, -1] # Minimize
NDIM = 2
N_CONSTRAINTS = 2
def SRN(individual):    
    x1=individual[0]
    x2=individual[1]
    objectives = ( (x1-2.0)**2 + (x2-1.0)**2+2.0,  9*x1-(x2-1.0)**2 )
    constraints = (225.0-x1**2-x2**2, -10.0 -x1 - 3*x2 )
    return (objectives, constraints)

F = SRN
X_RANGE = [0, 300]
Y_RANGE = [-300, 150]

# Setup toolbox

In [1]:
toolbox = nsga2.nsga2_toolbox(
    weights=WEIGHTS, n_constraints=N_CONSTRAINTS,
    bound_low=BOUND_LOW, bound_up=BOUND_UP)

NameError: name 'nsga2' is not defined

In [5]:
# Register test proglem as toolbox.evaluate
toolbox.register('evaluate', F)

In [38]:
from dask import delayed, compute

In [6]:
toolbox.population(2)[0].variable_labels
ind3 = creator.Individual([1, 2, 2])
ind3.variable_labels

['variable_0', 'variable_1']

In [43]:
from time import sleep
@delayed
def F2(x):
    sleep(1)
    return F(x)
F2([1,2])

Delayed('F2-78621636-3476-4088-b8af-d2897c618770')

In [44]:
mypop = toolbox.population(10)
toolbox.register('evaluate',F2)

In [45]:
%%time
evals = []
for ind in mypop:
    evals.append(toolbox.evaluate(ind))
compute(evals)

CPU times: user 4.62 ms, sys: 2.48 ms, total: 7.1 ms
Wall time: 2 s


([((2.0282865735541185, 0.9858520059082954),
   (4.028142242129252, -2.0717120225509023),
   (2.0282865735541185, 0.9858520059082954)),
  ((1.2719528607479513, 1.6404421403811262),
   (3.351770241161125, -1.3965194947740303),
   (1.2719528607479513, 1.6404421403811262)),
  ((1.5951349477387986, 1.1976157875891713),
   (3.0425039156778384, -1.185988340852521),
   (1.5951349477387986, 1.1976157875891713)),
  ((1.7020167851688954, 1.9144638153153146),
   (5.502959368987715, -2.945552236664123),
   (1.7020167851688954, 1.9144638153153146)),
  ((2.223041008904069, 0.6139951897010875),
   (4.3579117178438205, -2.4818652216401382),
   (2.223041008904069, 0.6139951897010875)),
  ((2.4671597479326226, 1.6157722871880271),
   (7.796509254913919, -4.61466527074314),
   (2.4671597479326226, 1.6157722871880271)),
  ((1.5387781061883066, 0.6245642232952215),
   (1.6585687671040827, -0.5945761996213065),
   (1.5387781061883066, 0.6245642232952215)),
  ((2.2363097912836922, 2.5176703321850864),
   (10

In [50]:
def main2(seed=None):
    random.seed(seed)

    NGEN = 50
    MU = 100
    CXPB = 0.9

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register('avg', np.mean, axis=0)
    stats.register('std', np.std, axis=0)
    stats.register('min', np.min, axis=0)
    stats.register('max', np.max, axis=0)
    
    logbook = tools.Logbook()
    logbook.header = 'gen', 'evals', 'std', 'min', 'avg', 'max'
    
    pop = toolbox.population(n=MU)

    nsga2.evaluate_population(toolbox, pop)

    # This is just to assign the crowding distance to the individuals
    # no actual selection is done
    pop = toolbox.select(pop, len(pop))
    
    record = stats.compile(pop)
    logbook.record(gen=0, evals=len(pop), **record)
    print(logbook.stream)

    # Begin the generational process
    for gen in range(1, NGEN):
        pop = nsga2.evolve_population(toolbox, pop, CXPB=CXPB)
        
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(pop), **record)
        #print(logbook.stream, hypervolume(pop, [1.0,1.0]))
        

    return pop, logbook

In [51]:
# Register test proglem as toolbox.evaluate
@delayed
def F2(x):
    return F(x)

toolbox.register('evaluate', F2)

In [52]:
%%time
    
pop, stats = main2()
pop.sort(key=lambda x: x.fitness.values)

print(stats)
#print("Convergence: ", convergence(pop, optimal_front))
#print("Diversity: ", diversity(pop, optimal_front[0], optimal_front[-1]))

import matplotlib.pyplot as plt
 
front = np.array([ind.fitness.values for ind in pop])
#optimal_front = numpy.array(optimal_front)
#plt.scatter(optimal_front[:,0], optimal_front[:,1], c="r")
plt.scatter(front[:,0], front[:,1], c="b")
plt.axis("tight")
plt.show()

TypeError: Both weights and assigned values must be a sequence of numbers when assigning to values of <class 'deap.creator.MyFitness'>. Currently assigning value(s) Delayed('getitem-a682cb2f4c6944c259e30bb5cd76f044') of <class 'dask.delayed.Delayed'> to a fitness with weights [-1, -1].

In [ ]:
# Recreate plots in Deb paper
fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(front[:,0], front[:,1], color='blue')
ax.set_xlim(X_RANGE)
ax.set_ylim(Y_RANGE)
ax.set_aspect('auto')
ax.set_title(NAME)

In [ ]:
[float(x) for x in pop[0]]

In [ ]:
pop[0].fitness.info

# Hypervolume

In [ ]:
from deap.benchmarks.tools import diversity, convergence, hypervolume

In [ ]:
print("Final population hypervolume is %f" % hypervolume(pop, [1.0,5.0]))